In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris

In [20]:
iris = load_iris()

data, target = iris["data"], iris["target"].reshape(1, -1)
print(f"data의 type : {type(data)},  data의 shape : {data.shape}")
print(f"target의 type : {type(target)},  target의 shape : {target.shape}")

data의 type : <class 'numpy.ndarray'>,  data의 shape : (150, 4)
target의 type : <class 'numpy.ndarray'>,  target의 shape : (1, 150)


In [21]:
np.unique(target, return_counts=True)

(array([0, 1, 2]), array([50, 50, 50], dtype=int64))

In [22]:
print(np.max(data), np.min(data))

# Normalize data to keep our gradients manageable
data = data / 7.9
print(np.max(data), np.min(data))

7.9 0.1
1.0 0.012658227848101266


In [23]:
# To build a zero-classifier, map target value of 0 into 1 and map all other target values into 0
target_new = np.zeros(target.shape)
target_new[np.where(target == 0)] = 1
target = target_new

In [24]:
np.unique(target, return_counts=True)

(array([0., 1.]), array([100,  50], dtype=int64))

In [25]:
# We'll split data into train and test set. We also transpose data to keep each example as a column.
m = 120
m_test = data.shape[0] - m

x_train, x_test = data[:m].T, data[m:].T
y_train, y_test = target[:, :m], target[:, m:]

print(f"x_train의 shape : {x_train.shape},   y_train의 shape : {y_train.shape}")
print(f"x_test의  shape : {x_test.shape},   y_test의 shape :  {y_test.shape}")

x_train의 shape : (4, 120),   y_train의 shape : (1, 120)
x_test의  shape : (4, 30),   y_test의 shape :  (1, 30)


In [26]:
# We'll shuffle the training set for a good measure
np.random.seed(138)
shuffle_index = np.random.permutation(m)
x_train, y_train = x_train[:, shuffle_index], y_train[:, shuffle_index]

In [27]:
def sigmoid(z):
    s = 1.0 / (1.0 + np.exp(-z))
    return s

In [28]:
def compute_loss(y, y_hat):
    m = y.shape[1]
    loss = -(1/m) * np.sum(np.multiply(y, np.log(y_hat)) + np.multiply((1-y), np.log(1-y_hat)))
    return loss

In [29]:
m = 60000
learning_rate = 1

X, Y = x_train, y_train
n_x, m = X.shape[0], X.shape[1]

W = np.random.randn(n_x, 1) * 0.01
b = np.zeros((1, 1))

Z = np.matmul(W.T, X) + b
A = sigmoid(Z)

cost = compute_loss(Y, A)

dW = (1/m) * np.matmul(X, (Y-A).T)
db = (1/m) * np.sum(Y-A)

W -= learning_rate * dW
b -= learning_rate * db

print(f"W의 shape : {W.shape}\nX의 shpae : {X.shape}\nb의 shape : {b.shape}\nZ의 shape : {Z.shape}\nA의 shape : {A.shape}\ncost의 value : {cost}")
print(f"dW의 shape : {dW.shape}\ndb의 shape : {db.shape}, Y의 shape {Y.shape}")

W의 shape : (4, 1)
X의 shpae : (4, 120)
b의 shape : (1, 1)
Z의 shape : (1, 120)
A의 shape : (1, 120)
cost의 value : 0.6913363486331121
dW의 shape : (4, 1)
db의 shape : (), Y의 shape (1, 120)
